In [1]:
from time import time
import numpy as np
from multiprocess import Pool
from itertools import chain

def nivelacion_cargas(D, n_p):
    s=len(D)%n_p
    t=int((len(D)-s)/n_p)
    out=[]
    for i in range(n_p):
        if i<s:
            out.append(D[i*t+i:i*t+i+t+1])            
        else:
            out.append(D[i*t+s:i*t+s+t])
    return out

def suma_Riemann(a, b, funcion, figuras_inscritas):
    t = time()
    ancho = (b - a) / figuras_inscritas
    print("Ancho de los rectangulos ->", ancho)
    suma_riemann = 0
    for i in range(1, figuras_inscritas + 1):
        datos = a + i * ancho
        suma_riemann += funcion(datos) * ancho
    tiempo_total = time() - t
    return suma_riemann, tiempo_total

def riemann_Pool(a, b, num_proce, funcion, figuras_inscritas):
    t = time()
    ancho = (b - a) / figuras_inscritas
    print("Ancho de los rectangulos ->", ancho)
    
    datos = np.linspace(a + ancho, b, figuras_inscritas)
    
    nivel_cargas = nivelacion_cargas(D=np.array(datos), n_p=num_proce)
    #print("Nivelacion de cargas ->", nivel_cargas)
    
    with Pool(num_proce) as pool:
        resul_sublistas = pool.map(funcion, nivel_cargas)
        
    #resultados_combinados =  [items for mini_lista in resul_sublistas for items in mini_lista] --> lo mismo que:
    resultados_combinados = list(chain.from_iterable(resul_sublistas))
    resul = sum(resultados_combinados) * ancho
    tiempo_total = time() - t
    return resul, tiempo_total

In [2]:
limite_inferior = 0
limite_superior = 5000
funcion = lambda x: x**2
num_procesadores = 4
figuras_inscritas =200

print("Sin usar Rienmann")
resultado, tiempo_total = suma_Riemann(a = limite_inferior,b= limite_superior, funcion=funcion,figuras_inscritas = figuras_inscritas)
print("tiempo Sin Pool ->", tiempo_total, "segundos")
print("Resultado ->", resultado, "\n")

print("Con Rienmann")
resultado, tiempo_total = riemann_Pool(a = limite_inferior,b= limite_superior, num_proce=num_procesadores, funcion=funcion,figuras_inscritas = figuras_inscritas)
print("tiempo CON Pool ->", tiempo_total, "segundos")
print("Resultado ->", resultado)

Sin usar Rienmann
Ancho de los rectangulos -> 25.0
tiempo Sin Pool -> 0.0 segundos
Resultado -> 41979687500.0 

Con Rienmann
Ancho de los rectangulos -> 25.0
tiempo CON Pool -> 0.7636711597442627 segundos
Resultado -> 41979687500.0


In [4]:
limite_inferior = 0
limite_superior = 40
funcion = lambda x:(x/2) + x**(1/2) + 2 
num_proce = 3
figuras_inscritas =400000

print("Sin usar Rienmann")
resultado, tiempo_total = suma_Riemann(a = limite_inferior,b= limite_superior, funcion=funcion,figuras_inscritas = figuras_inscritas)
print("tiempo Sin Pool ->", tiempo_total, "segundos")
print("Resultado ->", resultado, "\n")

print("Con Rienmann")
resultado, tiempo_total = riemann_Pool(a = limite_inferior,b= limite_superior, num_proce=num_proce, funcion=funcion,figuras_inscritas = figuras_inscritas)
print("tiempo CON Pool ->", tiempo_total, "segundos")
print("Resultado ->", resultado)

Sin usar Rienmann
Ancho de los rectangulos -> 0.0001
tiempo Sin Pool -> 0.26589012145996094 segundos
Resultado -> 648.6561245622664 

Con Rienmann
Ancho de los rectangulos -> 0.0001
tiempo CON Pool -> 0.7689282894134521 segundos
Resultado -> 648.6561245622644


In [21]:
limite_inferior = 0
limite_superior = 100000000
funcion = lambda x: x**2 + 2*x - x/2 * x**8 + 2*x**3
num_proce = 7
figuras_inscritas =100000000


print("Con Rienmann")
resultado, tiempo_total = riemann_Pool(a = limite_inferior,b= limite_superior, num_proce=num_proce, funcion=funcion,figuras_inscritas = figuras_inscritas)
print("tiempo Con Pool ->", tiempo_total, "segundos")
print("Resultado ->", resultado)


print("Sin usar Rienmann")
resultado, tiempo_total = suma_Riemann(a = limite_inferior,b= limite_superior, funcion=funcion,figuras_inscritas = figuras_inscritas)
print("tiempo Sin Pool ->", tiempo_total, "segundos")
print("Resultado ->", resultado, "\n")

Con Rienmann
Ancho de los rectangulos -> 1.0
tiempo Con Pool -> 27.062655925750732 segundos
Resultado -> -5.000000250000444e+78
Sin usar Rienmann
Ancho de los rectangulos -> 1.0
tiempo Sin Pool -> 99.16605043411255 segundos
Resultado -> -5.000000250000444e+78 

